In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import numpy as np
import time
from models import  LinfPGDAttack, ResNet18

In [2]:
learning_rate = 0.1
epsilon = 0.0314
k = 7
alpha = 0.00784
file_name = 'interpolated_adversarial_training'
mixup_alpha = 1.0

device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

In [3]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def mixup_data(x, y):
    lam = np.random.beta(mixup_alpha, mixup_alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).cuda()
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)


net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

adversary = LinfPGDAttack(net,epsilon,alpha,k)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)

In [5]:
def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    benign_loss = 0
    adv_loss = 0
    benign_correct = 0
    adv_correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)
        optimizer.zero_grad()

        benign_inputs, benign_targets_a, benign_targets_b, benign_lam = mixup_data(inputs, targets)
        benign_outputs = net(benign_inputs)
        loss1 = mixup_criterion(criterion, benign_outputs, benign_targets_a, benign_targets_b, benign_lam)
        benign_loss += loss1.item()

        _, predicted = benign_outputs.max(1)
        benign_correct += (benign_lam * predicted.eq(benign_targets_a).sum().float() + (1 - benign_lam) * predicted.eq(benign_targets_b).sum().float())
        if batch_idx % 10 == 0:
                print('\nCurrent batch:', str(batch_idx))
                print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
                print('Current benign train loss:', loss1.item())

        adv = adversary.perturb(inputs, targets)
        adv_inputs, adv_targets_a, adv_targets_b, adv_lam = mixup_data(adv, targets)
        adv_outputs = net(adv_inputs)
        loss2 = mixup_criterion(criterion, adv_outputs, adv_targets_a, adv_targets_b, adv_lam)
        adv_loss += loss2.item()

        _, predicted = adv_outputs.max(1)
        adv_correct += (adv_lam * predicted.eq(adv_targets_a).sum().float() + (1 - adv_lam) * predicted.eq(adv_targets_b).sum().float())
        if batch_idx % 10 == 0:
                print('Current adversarial train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
                print('Current adversarial train loss:', loss2.item())

        loss = (loss1 + loss2) / 2
        loss.backward()
        optimizer.step()

    print('\nTotal benign train accuarcy:', 100. * benign_correct / total)
    print('Total adversarial train accuarcy:', 100. * adv_correct / total)
    print('Total benign train loss:', benign_loss)
    print('Total adversarial train loss:', adv_loss)

In [6]:
def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    benign_loss = 0
    adv_loss = 0
    benign_correct = 0
    adv_correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            total += targets.size(0)

            outputs = net(inputs)
            loss = criterion(outputs, targets)
            benign_loss += loss.item()

            _, predicted = outputs.max(1)
            benign_correct += predicted.eq(targets).sum().item()

            if batch_idx % 10 == 0:
                print('\nCurrent batch:', str(batch_idx))
                print('Current benign test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
                print('Current benign test loss:', loss.item())

            adv = adversary.perturb(inputs, targets)
            adv_outputs = net(adv)
            loss = criterion(adv_outputs, targets)
            adv_loss += loss.item()

            _, predicted = adv_outputs.max(1)
            adv_correct += predicted.eq(targets).sum().item()

            if batch_idx % 10 == 0:
                print('Current adversarial test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
                print('Current adversarial test loss:', loss.item())

    print('\nTotal benign test accuarcy:', 100. * benign_correct / total)
    print('Total adversarial test Accuarcy:', 100. * adv_correct / total)
    print('Total benign test loss:', benign_loss)
    print('Total adversarial test loss:', adv_loss)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')

def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [7]:
t1 = time.time()
for epoch in range(0, 50):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)
t2 = time.time()


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.078125
Current benign train loss: 2.4463391304016113
Current adversarial train accuracy: 0.0
Current adversarial train loss: 4.894921779632568

Current batch: 10
Current benign train accuracy: 0.1640625
Current benign train loss: 3.1045103073120117
Current adversarial train accuracy: 0.125
Current adversarial train loss: 3.8991918563842773

Current batch: 20
Current benign train accuracy: 0.15625
Current benign train loss: 2.3740270137786865
Current adversarial train accuracy: 0.0859375
Current adversarial train loss: 2.553011417388916

Current batch: 30
Current benign train accuracy: 0.109375
Current benign train loss: 2.3121681213378906
Current adversarial train accuracy: 0.0859375
Current adversarial train loss: 2.4135799407958984

Current batch: 40
Current benign train accuracy: 0.15625
Current benign train loss: 2.2573416233062744
Current adversarial train accuracy: 0.1484375
Current adversarial train loss: 2.3


Current batch: 0
Current benign test accuracy: 0.41
Current benign test loss: 1.7651551961898804
Current adversarial test accuracy: 0.25
Current adversarial test loss: 2.1043882369995117

Current batch: 10
Current benign test accuracy: 0.45
Current benign test loss: 1.6949230432510376
Current adversarial test accuracy: 0.25
Current adversarial test loss: 2.0473289489746094

Current batch: 20
Current benign test accuracy: 0.34
Current benign test loss: 1.7303242683410645
Current adversarial test accuracy: 0.16
Current adversarial test loss: 2.0663161277770996

Current batch: 30
Current benign test accuracy: 0.38
Current benign test loss: 1.6998156309127808
Current adversarial test accuracy: 0.26
Current adversarial test loss: 2.078190803527832

Current batch: 40
Current benign test accuracy: 0.28
Current benign test loss: 1.8438541889190674
Current adversarial test accuracy: 0.13
Current adversarial test loss: 2.257385492324829

Current batch: 50
Current benign test accuracy: 0.35
Curr

Current adversarial train accuracy: 0.1875
Current adversarial train loss: 2.2137439250946045

Current batch: 310
Current benign train accuracy: 0.2421875
Current benign train loss: 2.0405778884887695
Current adversarial train accuracy: 0.1796875
Current adversarial train loss: 2.217130184173584

Current batch: 320
Current benign train accuracy: 0.3671875
Current benign train loss: 1.8771402835845947
Current adversarial train accuracy: 0.234375
Current adversarial train loss: 2.1055209636688232

Current batch: 330
Current benign train accuracy: 0.1171875
Current benign train loss: 1.573771595954895
Current adversarial train accuracy: 0.265625
Current adversarial train loss: 2.1158599853515625

Current batch: 340
Current benign train accuracy: 0.421875
Current benign train loss: 1.7752903699874878
Current adversarial train accuracy: 0.2890625
Current adversarial train loss: 2.108214855194092

Current batch: 350
Current benign train accuracy: 0.3828125
Current benign train loss: 1.942278


Current batch: 200
Current benign train accuracy: 0.328125
Current benign train loss: 1.947199821472168
Current adversarial train accuracy: 0.2265625
Current adversarial train loss: 2.156554698944092

Current batch: 210
Current benign train accuracy: 0.1328125
Current benign train loss: 1.7686123847961426
Current adversarial train accuracy: 0.140625
Current adversarial train loss: 2.184544563293457

Current batch: 220
Current benign train accuracy: 0.4140625
Current benign train loss: 1.8597347736358643
Current adversarial train accuracy: 0.1875
Current adversarial train loss: 2.1436679363250732

Current batch: 230
Current benign train accuracy: 0.5625
Current benign train loss: 1.717422366142273
Current adversarial train accuracy: 0.2421875
Current adversarial train loss: 2.127408266067505

Current batch: 240
Current benign train accuracy: 0.078125
Current benign train loss: 1.7712411880493164
Current adversarial train accuracy: 0.1875
Current adversarial train loss: 2.17478179931640

Current adversarial train accuracy: 0.140625
Current adversarial train loss: 2.220060348510742

Current batch: 100
Current benign train accuracy: 0.5078125
Current benign train loss: 1.643937587738037
Current adversarial train accuracy: 0.0703125
Current adversarial train loss: 2.023308277130127

Current batch: 110
Current benign train accuracy: 0.1484375
Current benign train loss: 1.85677170753479
Current adversarial train accuracy: 0.203125
Current adversarial train loss: 2.1371102333068848

Current batch: 120
Current benign train accuracy: 0.2890625
Current benign train loss: 2.0467772483825684
Current adversarial train accuracy: 0.1328125
Current adversarial train loss: 2.0311074256896973

Current batch: 130
Current benign train accuracy: 0.3671875
Current benign train loss: 2.0564563274383545
Current adversarial train accuracy: 0.2578125
Current adversarial train loss: 2.299757957458496

Current batch: 140
Current benign train accuracy: 0.53125
Current benign train loss: 1.5393505

...

Current adversarial train accuracy: 0.1328125
Current adversarial train loss: 1.7273650169372559

Current batch: 130
Current benign train accuracy: 0.1171875
Current benign train loss: 0.5706651210784912
Current adversarial train accuracy: 0.09375
Current adversarial train loss: 1.5644636154174805

Current batch: 140
Current benign train accuracy: 0.1015625
Current benign train loss: 0.39375045895576477
Current adversarial train accuracy: 0.53125
Current adversarial train loss: 1.450659990310669

Current batch: 150
Current benign train accuracy: 0.09375
Current benign train loss: 0.534870982170105
Current adversarial train accuracy: 0.390625
Current adversarial train loss: 1.9097741842269897

Current batch: 160
Current benign train accuracy: 0.671875
Current benign train loss: 1.537949800491333
Current adversarial train accuracy: 0.4296875
Current adversarial train loss: 1.6173505783081055

Current batch: 170
Current benign train accuracy: 0.0859375
Current benign train loss: 0.5925864


Current batch: 20
Current benign train accuracy: 0.0859375
Current benign train loss: 0.5304269790649414
Current adversarial train accuracy: 0.53125
Current adversarial train loss: 1.8134487867355347

Current batch: 30
Current benign train accuracy: 0.078125
Current benign train loss: 0.9243489503860474
Current adversarial train accuracy: 0.4140625
Current adversarial train loss: 1.8199355602264404

Current batch: 40
Current benign train accuracy: 0.9296875
Current benign train loss: 0.5750491619110107
Current adversarial train accuracy: 0.2734375
Current adversarial train loss: 1.808215618133545

Current batch: 50
Current benign train accuracy: 0.109375
Current benign train loss: 1.221298098564148
Current adversarial train accuracy: 0.1640625
Current adversarial train loss: 1.8035554885864258

Current batch: 60
Current benign train accuracy: 0.390625
Current benign train loss: 1.6386284828186035
Current adversarial train accuracy: 0.4921875
Current adversarial train loss: 1.711218237


Current batch: 20
Current benign test accuracy: 0.84
Current benign test loss: 0.5999542474746704
Current adversarial test accuracy: 0.38
Current adversarial test loss: 1.6702253818511963

Current batch: 30
Current benign test accuracy: 0.82
Current benign test loss: 0.6367045640945435
Current adversarial test accuracy: 0.4
Current adversarial test loss: 1.7035871744155884

Current batch: 40
Current benign test accuracy: 0.86
Current benign test loss: 0.6368045210838318
Current adversarial test accuracy: 0.43
Current adversarial test loss: 1.7218163013458252

Current batch: 50
Current benign test accuracy: 0.85
Current benign test loss: 0.5556038022041321
Current adversarial test accuracy: 0.43
Current adversarial test loss: 1.705235481262207

Current batch: 60
Current benign test accuracy: 0.82
Current benign test loss: 0.6443566083908081
Current adversarial test accuracy: 0.38
Current adversarial test loss: 1.8316612243652344

Current batch: 70
Current benign test accuracy: 0.76
Cur

Current adversarial train accuracy: 0.3359375
Current adversarial train loss: 1.9038076400756836

Current batch: 330
Current benign train accuracy: 0.40625
Current benign train loss: 1.6634156703948975
Current adversarial train accuracy: 0.1015625
Current adversarial train loss: 1.620795488357544

Current batch: 340
Current benign train accuracy: 0.828125
Current benign train loss: 0.7239537239074707
Current adversarial train accuracy: 0.2890625
Current adversarial train loss: 1.8345947265625

Current batch: 350
Current benign train accuracy: 0.2421875
Current benign train loss: 1.5625262260437012
Current adversarial train accuracy: 0.375
Current adversarial train loss: 1.5539697408676147

Current batch: 360
Current benign train accuracy: 0.1875
Current benign train loss: 1.6727385520935059
Current adversarial train accuracy: 0.140625
Current adversarial train loss: 2.0135350227355957

Current batch: 370
Current benign train accuracy: 0.2734375
Current benign train loss: 1.698225855827


Current batch: 220
Current benign train accuracy: 0.515625
Current benign train loss: 1.6566455364227295
Current adversarial train accuracy: 0.484375
Current adversarial train loss: 1.7587981224060059

Current batch: 230
Current benign train accuracy: 0.8828125
Current benign train loss: 1.2318586111068726
Current adversarial train accuracy: 0.4453125
Current adversarial train loss: 1.4699468612670898

Current batch: 240
Current benign train accuracy: 0.078125
Current benign train loss: 0.5352445840835571
Current adversarial train accuracy: 0.375
Current adversarial train loss: 1.9759643077850342

Current batch: 250
Current benign train accuracy: 0.90625
Current benign train loss: 0.6955764889717102
Current adversarial train accuracy: 0.453125
Current adversarial train loss: 1.810520887374878

Current batch: 260
Current benign train accuracy: 0.65625
Current benign train loss: 1.5535094738006592
Current adversarial train accuracy: 0.4609375
Current adversarial train loss: 1.8411644697

In [9]:
#RTX3070 50 epochs
import datetime
deltatime = str(datetime.timedelta(seconds=t2-t1))
print('Total running time is %s '% deltatime)

Total running time is 4:38:57.181549 
